In [21]:
import numpy as np
import math
import time

#----------------------------------------------EDITABLE VARIABLES----------------------------------------------------

# Number of simulations that run in order to be averaged together.
# More runs will lead to a longer load time. 
RUNS = 110

# Number of maximum generations per simulation.
MAXGEN = 100

# This sets the number of strategies in the game.
# It should always be the same as the number of rows in the payoff matrix.
TYPES = 2

# Mutation Probability (set to zero for no mutation)
mu = 0.01

# Payoff Matrix (3 strategies)
# Symmetric games only - payoffs listed are only for player 1
matrix = np.array([[2,0],
                   [3,1]])

# Correlation parameter r - change to increase correlation or anti-correlation.
# Between 0 and 1: correlation 
# Between 0 and -1: anti-correlation
r = 0.00

#------------------------------------------------EDIT WITH CAUTION---------------------------------------------------

# Initialize the variables
pop = np.zeros(TYPES)
oldpop = np.zeros(TYPES)
stable = 0
totals = np.zeros(TYPES)
results = np.zeros((MAXGEN, TYPES))
sum_proportions = np.zeros(TYPES)

# Get a random (flat) starting distribution
def rand_fill(TYPES):
    x = 0
    while x == 0:
        random = np.random.rand(TYPES)
        y = 0 - np.log(random)
        x = np.sum(y)
    pop = y / x
    return pop

# The discrete-time replicator dynamics, with correlation* and mutation
def replicate(pop, oldpop, TYPES, mu, matrix, r):
    # Clear payoffs
    fitness = np.zeros(TYPES)
    
    # Perform fitness calculations
    for i in range(TYPES):
        for j in range(TYPES):
            payoff_ij = matrix[i][j] * pop[j]  # The existing payoff
            correlated_payoff = matrix[i][j]  # Your correlated payoff based on payoff matrix
            
            # Apply the formula for the correlated payoff
            fitness[i] += r * correlated_payoff + (1 - r) * payoff_ij
    
    avefit = np.sum(fitness * pop)
    new_pop = np.zeros(TYPES)

    # Replicate based on fitness
    for i in range(TYPES):
        new_pop[i] = pop[i] * ((1 - mu) * fitness[i] / avefit) + mu / TYPES
    return new_pop

# Detect a  stable state based on a difference threshold
def detect_stable(oldpop, newpop, threshold=1e-5):
    return np.all(np.abs(oldpop - newpop) < threshold)

# Print results of run
def print_run(gen):
    global totals
    print(" ".join(map(str, pop)), " Gen: ", gen)
    totals += pop

def print_run(gen, pop, totals):
    print(" ".join(map(str, pop)), "Gen: ", gen)
    return totals + pop

# Prints the mean proportions across the simulations
# You can edit the number in "{:.3f}" to change the decimal point the print function truncates to
def print_results(sum_proportions, RUNS):
    mean_proportions = sum_proportions / RUNS
    formatted_proportions = ["{:.3f}".format(prop) if prop >= 0.0001 else "0.00" for prop in mean_proportions]
    print("\033[1m\nMean Proportions Across Simulations: \033[0m")
    print(" ".join(formatted_proportions))

# Main program
def main(RUNS, MAXGEN, TYPES, mu, matrix, r):
    # Ensure different seeds for each run
    np.random.seed(int(time.time()))
    sum_proportions = np.zeros(TYPES)
    totals = np.zeros(TYPES)
    
    # Performs each simulation run
    for _ in range(RUNS):
        gen = 0
        pop = rand_fill(TYPES)
        stable = False

        # Runs the simulation until it's stable or reaches the maximum number of generations
        while not stable and gen < MAXGEN:
            oldpop = np.copy(pop)
            newpop = replicate(pop, oldpop, TYPES, mu, matrix, r)
            stable = detect_stable(oldpop, newpop)
            pop = newpop
            gen += 1

        # Add 
        totals = print_run(gen, pop, totals)
        sum_proportions += pop
        
    # Print full results
    print_results(sum_proportions, RUNS)

main(RUNS, MAXGEN, TYPES, mu, matrix, r)

0.005050255636222095 0.9949497443637778 Gen:  7
0.005050314417328676 0.9949496855826715 Gen:  10
0.0050502799665679485 0.9949497200334322 Gen:  7
0.005050317336455271 0.9949496826635446 Gen:  5
0.0050502539746382965 0.9949497460253616 Gen:  6
0.005050257685405778 0.9949497423145941 Gen:  8
0.005050251392411911 0.9949497486075882 Gen:  10
0.0050502995146529655 0.994949700485347 Gen:  7
0.005050262996281422 0.9949497370037184 Gen:  5
0.005050339006271992 0.994949660993728 Gen:  7
0.005050445132003517 0.9949495548679963 Gen:  4
0.005050266089532813 0.9949497339104674 Gen:  5
0.005050302948304268 0.9949496970516956 Gen:  4
0.005050358607833738 0.9949496413921662 Gen:  4
0.005050272674025125 0.9949497273259749 Gen:  10
0.005050255441433285 0.9949497445585667 Gen:  7
0.005050250954488353 0.9949497490455117 Gen:  7
0.005050310189493166 0.9949496898105066 Gen:  8
0.005050382652463348 0.9949496173475365 Gen:  6
0.005050252547678147 0.994949747452322 Gen:  10
0.005050265820014138 0.9949497341799

In [24]:
# Correlation: 
# rπ(i,j) + (1-r)Σjπ(i,j)xj

In [25]:
# Anti-correlation: 
# Σjπ(i,j)xj + (1-r)π(i,j)